In [79]:
#Import/Install Packages
import datetime
import os
import requests
import time
import json
      
!pip install --quiet geopandas
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline
import numpy as np
requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)

In [80]:
#Prep for API Call
# Set root URL for API requests
root_url = 'https://api.climateengine.org/'
# Authentication info for the API (INSERT YOUR OWN KEY)
headers = {'Authorization': 'INSERT YOUR API KEY HERE'}
     

In [ ]:
#Initialize Earth Engine to use AOI Asset 
# import Google earth engine module
import ee
ee.Authenticate()
#Authenticate the Google earth engine with google account
ee.Initialize()

In [ ]:
# Mount Google Drive for exporting CSVs + PNGs
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Create list of aoi names (INSERT YOUR OWN ASSET OF INTEREST AND UPDATE PROPERTY TO LOOP THROUGH)
#Reminder - Make sure to give "anyone can read" access to your asset 
aoi_names = ee.FeatureCollection("projects/ee-kristenoshea/assets/fieldAOI").aggregate_array("FieldID").getInfo()
print(aoi_names)

In [ ]:
#Write Function 
def write_openet_timeseries(aoi):
  
  #Set Dataset Name as Variable
  dataset = "OPENET_CONUS"

  print("Making Request for " + str(aoi))

  #Make a list of ET Model Options
  models = ['et_eemetric','et_ssebop','et_sims','et_geesebal','et_ptjpl','et_disalexi']

  #Make an empty list
  series = []

  #Climate Engine API Point 
  endpoint = 'timeseries/native/custom_asset'

  #Define Parameters
  for variable in models:
    params = {
      'dataset': dataset,
      'variable': variable,
      'start_date': '2016-01-01',
      'end_date': '2021-12-01',
      'asset_id': 'projects/ee-kristenoshea/assets/fieldAOI',
      'area_reducer': 'mean',
      'filter_by': 'FieldID',
      'sub_choices': json.dumps(aoi)
     }
    # Send request to the API 
    r = requests.get(root_url + endpoint, params=params, headers=headers, verify=True)
    response = r.json()
    #print(response)
    
    #response (may need to unpack with [] around timeseries the first time)
    [timeseries] = response

    #Select Data
    data = timeseries['Data']
    
    #Convert to Data Frame
    df = pd.DataFrame(data)
    
    #Select as Series
    date = df.iloc[:,0]
    value = df.iloc[:,1]

    #Append series to blank list
    series.append(date)
    series.append(value)

  #Create Dataframe from List 
  DF = pd.DataFrame(series)

  #Drop Duplicate Date Columns
  DF = DF.T.loc[:,~DF.T.columns.duplicated()]

  print(DF)

  #Format dates for plotting
  DF['Date'] = pd.to_datetime(DF['Date'])

  #Date Value for Plottig
  date_plot = DF['Date']

  #Model Values
  et_eemetric = DF["et_eemetric (mm)"]
  et_ssebop = DF["et_ssebop (mm)"]
  et_sims = DF["et_sims (mm)"]
  et_geesebal = DF["et_geesebal (mm)"]
  et_ptjpl = DF["et_ptjpl (mm)"]
  et_disalexi = DF["et_disalexi (mm)"]

  #Generate graph
  fig, ax = plt.subplots(figsize=(18,6),facecolor='white')
  name = str(aoi)
  ax.plot(date_plot, et_eemetric, label = 'et_eemetric')  
  ax.plot(date_plot, et_ssebop, label = 'et_ssebop') 
  ax.plot(date_plot, et_sims, label = 'et_sims') 
  ax.plot(date_plot, et_geesebal, label = 'et_geesebal') 
  ax.plot(date_plot, et_ptjpl, label = 'et_ptjpl') 
  ax.plot(date_plot, et_disalexi, label = 'et_disalexi') 

  #Add x-axis label
  plt.xlabel("Dates")
                      
  #Add y-axis label
  plt.ylabel('mm')
  plt.ylim(0,250)
           
  #Add Title
  plt.title('ET Models Demo', fontsize = 16)

  #Add Legend 
  plt.legend(bbox_to_anchor= (1.1, 1.1),loc='upper right', borderaxespad=0)

  #Show
  plt.show()

  print("Exporting CSV for " + str(aoi))
  DF.to_csv("/content/drive/MyDrive/API_Testing/" + str(aoi) + "_" + dataset + ".csv")
  plt.savefig("/content/drive/MyDrive/API_Testing/" + str(aoi) + "_" + dataset + ".png")

In [ ]:
# Loop over allotment names to timeseries CSVs 
for i in aoi_names:
  write_openet_timeseries(i, dataset)